In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from yolov3_tf2.models import YoloV3
from yolov3_tf2.utils import draw_outputs

# Load the pre-trained YOLOv3 model
yolo = YoloV3(classes=80)
yolo.load_weights('./checkpoints/yolov3.tf').expect_partial()

# Helper function to preprocess the image for YOLOv3
def preprocess(image):
    image = tf.image.resize(image, (416, 416))
    image = image / 255.0
    image = image[tf.newaxis, ...]
    return image

# Load and preprocess the image
image_path = tf.keras.utils.get_file(
    'YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image_raw = tf.io.read_file(image_path)
image_raw = tf.image.decode_image(image_raw)

image = preprocess(image_raw)

# Get predictions
pred = yolo(image)
boxes, scores, classes, nums = pred

# Helper function to decode predictions
def decode_predictions(image, boxes, scores, classes, nums):
    image = np.array(image)
    for i in range(nums[0]):
        x1y1 = tuple((np.array(boxes[0][i][:2]) * [image.shape[1], image.shape[0]]).astype(np.int32))
        x2y2 = tuple((np.array(boxes[0][i][2:]) * [image.shape[1], image.shape[0]]).astype(np.int32))
        image = cv2.rectangle(image, x1y1, x2y2, (255, 0, 0), 2)
        image = cv2.putText(image, '{} {:.4f}'.format(classes[0][i], scores[0][i]),
                            x1y1, cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
    return image

# Draw original predictions
image_with_bboxes = draw_outputs(image_raw.numpy(), (boxes, scores, classes, nums), class_names)

plt.figure(figsize=(8, 8))
plt.imshow(image_with_bboxes)
plt.title('Original Image with YOLOv3 Detections')
plt.show()

# Loss object
loss_object = tf.keras.losses.CategoricalCrossentropy()

# Create adversarial pattern
def create_adversarial_pattern(input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = yolo(input_image)
        loss = loss_object(input_label, prediction)

    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    return signed_grad

# Dummy label for the sake of the example
label = tf.constant([[0, 0, 0, 0, 1]])  # This should be the one-hot encoded label

perturbations = create_adversarial_pattern(image, label)
plt.imshow(perturbations[0] * 0.5 + 0.5)

# Display adversarial examples
def display_images(image, description):
    pred = yolo(image)
    boxes, scores, classes, nums = pred
    image_with_bboxes = draw_outputs(image_raw.numpy().copy(), (boxes, scores, classes, nums), class_names)
    plt.figure(figsize=(8, 8))
    plt.imshow(image_with_bboxes)
    plt.title(description)
    plt.show()

epsilons = [0, 0.01, 0.1, 0.15]
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input') for eps in epsilons]

for i, eps in enumerate(epsilons):
    adv_x = image + eps * perturbations
    adv_x = tf.clip_by_value(adv_x, 0, 1)
    display_images(adv_x, descriptions[i])


ModuleNotFoundError: No module named 'yolov3_tf2'